In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

i = 5

ex_users = pd.read_parquet(f"group_{i}_user_data.parquet")
ex_user_reviews = pd.read_parquet(f"group_{i}_user_reviews.parquet")

ex_users = users[users.num_ratings != 0]
ex_user_reviews = user_reviews[user_reviews.rating != 0]

print(len(ex_users))
print(len(ex_user_reviews))

users.head(3)

361
12099


,user_url,user_id,name,num_ratings,avg_rating,num_reviews,is_best_reviewer,reviewer_rank,is_most_followed,follow_rank
0,https://www.goodreads.com/user/show/2111327-tori,2111327-tori,Tori,2255,3.74,29,False,0,False,0
1,https://www.goodreads.com/user/show/156563474-...,156563474-annabel,Annabel,219,3.32,219,False,0,False,0
2,https://www.goodreads.com/user/show/16861918-s...,16861918-sailakshmi-deepak,Sailakshmi Deepak,377,3.62,3,False,0,False,0


#### Full df

In [54]:
original_users = pd.read_parquet(f"users_data.parquet")
old_length = len(original_users)
original_users = original_users[original_users.num_ratings != 0]
print(f"There are {len(original_users)} original users ({len(original_users)*100/old_length:.2f}%)")

original_users_reviews = pd.read_parquet(f"users_40_book_reviews.parquet")
old_length = len(original_users_reviews)
original_user_reviews = original_users_reviews[original_users_reviews.rating != 0]
print(f"There are {len(original_user_reviews)} new user reviews ({len(original_user_reviews)*100/old_length:.2f}%)")


There are 9466 original users (64.75%)
There are 330180 new user reviews (92.66%)


In [55]:
original_users_reviews.head()

,user_id,title_id,title,rating,votes
0,7869133-sarah,11389341-the-meaning-of-marriage,The Meaning of Marriage: Facing the Complexiti...,3,13
1,7869133-sarah,116789.Himalayan_Quest,"Himalayan Quest: Ed Viesturs on the 8,000-Mete...",4,2
2,7869133-sarah,214614.Desert_Solitaire,Desert Solitaire,4,2
3,7869133-sarah,37566.Trapped_in_the_Mirror,Trapped in the Mirror: Adult Children of Narci...,3,1
4,7869133-sarah,40265755-annapurna,"Annapurna: The First Conquest of an 8,000-Mete...",4,1


In [70]:
len(original_users_reviews)

356348

#### Get title ids from top 

In [81]:
top_5_reviews_for_each_user = original_users_reviews.groupby('user_id').head(5)

In [92]:
top_5_reviews_for_each_user.head()

,user_id,title_id,title,rating,votes
0,7869133-sarah,11389341-the-meaning-of-marriage,The Meaning of Marriage: Facing the Complexiti...,3,13
1,7869133-sarah,116789.Himalayan_Quest,"Himalayan Quest: Ed Viesturs on the 8,000-Mete...",4,2
2,7869133-sarah,214614.Desert_Solitaire,Desert Solitaire,4,2
3,7869133-sarah,37566.Trapped_in_the_Mirror,Trapped in the Mirror: Adult Children of Narci...,3,1
4,7869133-sarah,40265755-annapurna,"Annapurna: The First Conquest of an 8,000-Mete...",4,1


In [93]:
top_5_reviews_for_each_user.to_parquet('top_5_reviews_for_each_original_user.parquet')

In [82]:
title_ids = top_5_reviews_for_each_user.title_id.unique().tolist()
print(f"There are {len(title_ids)} unique titles from these users ({len(title_ids)*100/len(top_5_reviews_for_each_user):.1f}%)")

There are 28808 unique titles from these users (63.5%)


In [83]:
base_url = 'https://www.goodreads.com/book/show/'
full_title_urls = [base_url + i for i in title_ids]
full_title_urls[:5]

['https://www.goodreads.com/book/show/11389341-the-meaning-of-marriage',
 'https://www.goodreads.com/book/show/116789.Himalayan_Quest',
 'https://www.goodreads.com/book/show/214614.Desert_Solitaire',
 'https://www.goodreads.com/book/show/37566.Trapped_in_the_Mirror',
 'https://www.goodreads.com/book/show/40265755-annapurna']

In [84]:
len(full_title_urls)

28808

#### Writing files

In [90]:
def write_urls_to_chunks(url_list, chunk_size=500, file_name_prefix='', output_dir='.'):

    os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

    for i in range(0, len(url_list), chunk_size):
        chunk = url_list[i:i + chunk_size]
        file_index = (i // chunk_size) + 1
        full_path = os.path.join(output_dir, f'{file_name_prefix}_{file_index}.txt')
        
        with open(full_path, 'w') as f:
            f.writelines(url + '\n' for url in chunk)


In [91]:
write_urls_to_chunks(full_title_urls, file_name_prefix = 'new_titles_group', output_dir = 'new_titles')